# Alpha–Phi Binary Explorer (v6)

**New tweaks (v5 → v6)**  
1. **Rational depth map** `F_rational` with divisor `(1 + c·√n)` to avoid hard log jumps.  
2. **Stronger adaptive feedback** (`gamma` defaults 0.05) and lower `p,q` defaults to shift fixed point.  
3. **Extended hybrid controls** – `phi_scale` scans 0.75–0.95, `comp_p` scans 1.3–1.7.  
4. **Residence‑time & band‑ladder demo** – deeper seed (4096 bits) and 400 iterations.  
5. **Wider parameter scan**: `p ∈ {0.6,0.8,1.0,1.2}`, `q ∈ {0.9,1.1,1.3}`, `lam ∈ {0.02,0.04,0.06}`, `gamma ∈ {0.03,0.05}`, `phi_scale ∈ {0.75,0.8,0.85,0.9,0.95}`; steps = 30.  
6. **Early‑exit success flag** – prints first hit < 1 % and stops further search to save Colab time.

Increase `mp.mp.dps` if you chase ppm‑level convergence.


In [1]:
import mpmath as mp, csv, matplotlib.pyplot as plt, collections, math, itertools, sys
mp.mp.dps = 300  # extra headroom

ALPHA = mp.mpf('7.2973525693e-3')
PHI   = (1 + mp.sqrt(5)) / 2

def fib_word_bits(n):
    a, b = '0', '01'
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fib_word_real(n):
    bits = fib_word_bits(n)
    return mp.fsum(int(b) * mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


## Rational Depth Map Test

In [2]:
# --- Rational depth map ---
def F_rational(x, n_bits, p=1.5, c=0.015):
    return 1 / (x**p * (1 + c*mp.sqrt(n_bits)))

print('F_rational quick scan')
for c in (0.01,0.015,0.02):
    for n in (64,128,256,512,1024,2048):
        err = abs((F_rational(fib_word_real(n), n, p=1.5, c=c)-ALPHA)/ALPHA)
        if n==64: print(f'c={c}')
        print(f' n={n:<4} rel_err={mp.nstr(err,4)}')
    print()


F_rational quick scan
c=0.01
 n=64   rel_err=810.7
 n=128  rel_err=786.5
 n=256  rel_err=754.7
 n=512  rel_err=713.8
 n=1024 rel_err=663.1
 n=2048 rel_err=602.5

c=0.015
 n=64   rel_err=781.7
 n=128  rel_err=748.4
 n=256  rel_err=705.9
 n=512  rel_err=653.5
 n=1024 rel_err=591.3
 n=2048 rel_err=521.1

c=0.02
 n=64   rel_err=754.7
 n=128  rel_err=713.8
 n=256  rel_err=663.1
 n=512  rel_err=602.5
 n=1024 rel_err=533.5
 n=2048 rel_err=459.1



## Adaptive‑bias Iterate (γ=0.05)

In [3]:
# --- Adaptive‑bias iterate w/ stronger gamma ---
def iterate_adapt(x0, steps=30, p=1.0, q=1.3, lam=0.05, gamma=0.05):
    traj=[x0]
    for _ in range(steps):
        eta  = gamma * (x0/ALPHA - 1)
        core = 1 / ((10**(q+eta)) * x0**p)
        x0   = core**lam
        traj.append(x0)
    return traj

seed = fib_word_real(512) * mp.sqrt(2)
traj = iterate_adapt(seed)
print('k  rel_err')
for k,val in enumerate(traj):
    if k%6==0:
        print(k, mp.nstr(abs((val-ALPHA)/ALPHA),4))


k  rel_err
0 55.24
6 76.58
12 76.93
18 76.94
24 76.94
30 76.94


## Tuned Hybrid Ping‑Pong Sample

In [4]:
# --- Tuned hybrid ping‑pong (scannable) ---
def hybrid_tuned(x0, cycles=12, q=2.0, comp_p=1.5, phi_scale=0.85):
    traj=[x0]
    for _ in range(cycles):
        x0 = (phi_scale*PHI) / x0
        x0 = 1 / (10**q * x0**comp_p)
        traj.append(x0)
    return traj

seed = fib_word_real(256)
traj = hybrid_tuned(seed,q=2.0,comp_p=1.4,phi_scale=0.8)
print('cycle  rel_err')
for k,val in enumerate(traj):
    print(k, mp.nstr(abs((val-ALPHA)/ALPHA),4))


cycle  rel_err
0 38.77
1 0.8311
2 0.9999
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0


## Residence‑Time Histogram for 4096‑bit Seed

In [5]:
# --- Residence‑time / ladder test ---
def band_index(x):
    if x < ALPHA: return -1
    ratio = x/ALPHA
    return int(math.floor((ratio-1)*10))   # 0→ [1,1.1), 1→[1.1,1.2)...

deep_seed = fib_word_real(4096)
traj = iterate_adapt(deep_seed, steps=400)
band_counts = collections.Counter(band_index(v) for v in traj)
print('Band index : count (−1 = below α)')
for idx in range(-1,8):
    print(idx, band_counts.get(idx,0))


Band index : count (−1 = below α)
-1 0
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0


## Jacobian / Lyapunov Check

In [6]:
# --- Lyapunov diagnostic ---
def jacobian(x, p,q,lam,gamma):
    eta = gamma*(x/ALPHA-1)
    core = 1 / ((10**(q+eta)) * x**p)
    return lam * core**(lam-1) * (-p/x - math.log(10)*(q+eta))

series = iterate_adapt(fib_word_real(512), steps=40, p=1.0, q=1.3, lam=0.05, gamma=0.05)
Js = [jacobian(series[i],1.0,1.3,0.05,0.05) for i in range(-10,-1)]
lyap = sum(math.log(abs(J)) for J in Js)/len(Js)
print('Mean |J| last 9 steps =', sum(abs(J) for J in Js)/9)
print('Lyapunov exponent ≈', lyap)


Mean |J| last 9 steps = 30867.3095218796565897815298531904802136132704346423713593849443244528812795711346154414744390976310348903591792710272739085261850871437488905968411652708237865868016173621230535051417160320266527630358064179873642556201389206314104327239867703365282328289228965979788717184679958128107968371638926737
Lyapunov exponent ≈ 10.33745295859204


## Expanded Parameter Sweep (stops at first <1 % hit)

In [8]:
# --- Huge parameter sweep until first success ---
param_space = list(itertools.product(
    (0.6,0.8,1.0,1.2),        # p
    (0.9,1.1,1.3),            # q
    (0.02,0.04,0.06),         # lam
    (0.03,0.05),              # gamma
    (0.75,0.8,0.85,0.9,0.95), # phi_scale
    (256,512,1024)            # bits
))
print('Scanning', len(param_space), 'combos…')
for (p,q,lam,gamma,phi_scale,bits) in param_space:
    seed = fib_word_real(bits)
    x = iterate_adapt(seed,30,p,q,lam,gamma)[-1]
    x = hybrid_tuned(x,cycles=8,q=q,comp_p=p,phi_scale=phi_scale)[-1] # Extract the last element
    rel = abs((x-ALPHA)/ALPHA)
    if rel < 0.01:
        print('HIT!', p,q,lam,gamma,phi_scale,bits,'rel',rel)
        break
else:
    print('No hit <1% in search box')

Scanning 1080 combos…
No hit <1% in search box
